In [1]:
from functions import *

In [2]:
regions_dict = {'nom':'BT5137', 'details':'BT5138', 'num_mat':'BT5139', 'signalement':'BT5140',
                'affectation':'BT5141', 'instruction':'BT5142', 'adresse':'BT5143', 'dates':'BT5144', 
                'etat_civil':'BT5145', 'decision':'BT5146'}

In [3]:
for key, value in regions_dict.items():
    print(key + " : " + value)

nom : BT5137
details : BT5138
num_mat : BT5139
signalement : BT5140
affectation : BT5141
instruction : BT5142
adresse : BT5143
dates : BT5144
etat_civil : BT5145
decision : BT5146


In [4]:
directory = "/Users/Theo/Desktop/big_drop/"

In [5]:
output = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        # Load the XML file into an ElementTree object
        tree = ET.parse(os.path.join(directory, filename))
        
        result = {}
        for key, value in regions_dict.items():
            line1 = ".//{http://www.loc.gov/standards/alto/ns-v4#}"
            line2 = f"TextBlock[@TAGREFS='{value}']/"
            lines = tree.findall(line1 + line2 + "{http://www.loc.gov/standards/alto/ns-v4#}TextLine")
        
            cat = []
        
            for content in lines:
                cat.append(content.find('.//{http://www.loc.gov/standards/alto/ns-v4#}String').attrib['CONTENT'])
            
            result[f'{key}'] = cat
            df_dictionary = pd.DataFrame([result])
            
        output = pd.concat([output, df_dictionary], ignore_index=True)

In [6]:
output['instruction_militaire'] = output['instruction'].apply(extract_intruction_militaire)
output['instruction'] = output['instruction'].apply(extract_digit_or_x)
output['taille'] = output['signalement'].apply(extract_height)
output['prenom'], output['nom'] = zip(*output['nom'].apply(extract_first_name))
output['date_naissance'] = output['etat_civil'].apply(extract_date)
output['num_mat'], output['classe'] = zip(*output['num_mat'].apply(lambda x: (x, extract_classe(x))))
output['num_mat'] = output['num_mat'].apply(clean_num_mat)
output['details'] = output['details'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)



In [7]:
order = ['nom', 'prenom', 'date_naissance', 'num_mat', 'classe', 'signalement', 'taille', 
         'instruction', 'instruction_militaire', 'details', 'affectation', 'adresse', 
         'dates', 'etat_civil', 'decision']

output = output[order]

In [8]:
output.sample(n=40)

,nom,prenom,date_naissance,num_mat,classe,signalement,taille,instruction,instruction_militaire,details,affectation,adresse,dates,etat_civil,decision
44,Bréart,Marcel,4 fevrier 1889,84,1909.0,[],NaN,None,None,du 10^e Arrt85Incorpé au 152^e Regt d'Infanter...,"[152^e Regt d'Infanterie 3694, 94^e Rég^t d'I...","[Etat 39. Gueschart, C^on de Crécy (Somme) R,...",[],"[Paris, 4 Fevrier 1889, e la Seine, u 10^e arr...","[1, 10]"
57,Robin,Auguste,19 juillet 1887,2918,NaN,"[et, bruns, bruns, ordinaire, droit, moyenne, ...",159.0,3,None,Dirigé le 1 octobe 1908 sur le 19^e Regt de C...,"[19^e Regt de Chasseurs, 3e Regt Chasseurs 309...","[Etat d'aff^on 20^bis rue Caillaux 13^e, 1-5-1...","[OCT 1 OCT 1 OCT 1 OCT, 1921 1927 1933]","[Gentilly, 19 Juillet 1887, e la Seine, Villej...","[1, 08]"
72,Poblin,Louis Didier Lucien,28 decembre 1893,2035,1911.0,"[Monas, bleus, moyen, ordinaire, ovale, 63]",163.0,3,None,Incorporé à compter du 28 Novembre 191. Arri...,"[4e Régiment d'Infanterie 5529, 26^e B^on de ...","[g 29.3.20 Paris 180 R de Charenton - 9.2, G^i...",[],"[28 decembre 1893, Paris, d 14^e Arrondissem, ...","[1096, de 11 Arrondissement, 1, 13]"
56,Maréchal,Henri Lévis,21 décembre 1874,1107,1892.0,"[et, bruns, gris, ordinaire, long, moyenne, ro...",166.0,x,None,Engagé Volre pour 3 ans le 5 Novembre 1893 a A...,"[10e Cie d'ouv d'artie, 26e d'artillerie, Rég ...","[17 8bre 97 Aubervilliers, 3 rue Crévet, k, 28...","[5 9bre 5 9bre 5 9bre 5 9bre, 1896 1906 1912 1...","[21 Décembre 1874,, Aubervilliers, u dit, e l...","[Engagé Volontaire pour 3ans, 4]"
47,Tetu,Victor,11 sept. 1872,4818,NaN,[69],169.0,3,None,Casier Judiciaire Neant.X Desaffecté du 25^e T...,[25e Regt 11 579 278],[],[],"[11 Sept. 1872, Arcueil - Cachan, Villejuif, e...",[Exempté : Bronchite chronique. -]
19,Dany,"Alfred, Gilbert",14 janvier 1879,2814,1897.0,"[noirs, ovale, noirs, haut, 69, moyen]",169.0,3,None,Engagé Volontaire pour 4 ans le 17 Janvier 189...,"[146^e Régt dinfanterie -, 11^e Batl^on de Ch...","[10 Janvier 1903 Paris 16 Rue de la Chapelle, ...",[],"[Paris, 14 Janvier 1879, e la Seine, u 18^e ar...","[Engagé Volontaire, 490, u 18^e Arrondiss^t de..."
50,Lallice,"Joseph, François",11 décembre 1876,3009,NaN,[],NaN,3,None,Déclaré insoumis le 2 Janvier 1898 N 5154.Ray...,[],[],[1^e 9^br1900 1^e 9^br1910 1er 9^br 1916 1^e 9...,"[11 décembre 1876, Paris, u 14^e arrond^t, e ...","[Bon absent, 1]"
89,Hebbelynck,Mareel André,8 novembre 1897,1517,NaN,"[bruns, allongé, d°, droit, vexe, 75]",175.0,3,None,Incorporé le 11 Janvier 1916 services comptant...,"[162^e Régiment d'Infanterie 14450, 9^e Régi...",[C° rgt 11.3.24 Paris 5 Rue Duguesclin],[],"[Paris, 8 Novembre 1897, da la seine, u 15, u ...","[646, u 15e Arrodissement, 1, 15]"
63,Delinotte,Ernest,18 mars 1868,530,1888.0,"[d, chat. foncé, ronds, ordinaire, moyen, moye...",163.0,3,exerce,le 148^e Regiment d'Inf^ie 13 novembre 1889Ca...,"[148^e de ligne, Régt d'Infanterie, a Beauvai...","[8 Août 92 Paris 64 Ru la Glacière, E.A, 13 8^...","[1^er Octobre 1^er Octobre, 1892 1849 1905 191...","[18 mars 1868, Paris, u 13^e arrt, e la seine,...","[Bon service armé, 1, 1]"
96,Gesmier,"Jean, Baptiste, Auguste, Constant",11 février 1879,3994,NaN,"[et, Châtains clair, bleus, large, moyen, moy...",166.0,3,None,Dirigé le 1novembre 1900 sur le. 151^e régime...,"[151^e régiment d'Infanterie, CM 22^c S^on c...","[9 rue de Cheroy 17, 26/2/199 19 Ru de Chèroy...",[1^er Nov^bre 1^er Nov^bre 1^er Nov^bre 1^er N...,"[Ducey, 11 février 1879, la Manche, udit, Par...","[Bon pour le service armé, 1ère]"


In [9]:
output.to_excel('/Users/Theo/Desktop/sample_table.xlsx', index=False)

In [10]:
output.to_csv('/Users/Theo/Desktop/sample_table.tsv', sep = '\t', index=False)